In [1]:
import jax.numpy as jnp
from jax import custom_jvp
from jax import custom_vjp
from jax import grad
import numpy as np
from scipy.special import wofz

In [58]:
@custom_jvp
def f(x, y):
    return wofz(x+(1j)*y).real

@f.defjvp
def f_jvp(primals, tangents):
    x, y = primals
    x_dot, y_dot = tangents
    primal_out = f(x, y)
    V=primal_out
    L=wofz(x+(1j)*y).imag
    dVdx = 2.*(y*L - x*V)
    dVdy = 2.*(x*L + y*V) - 2./np.sqrt(2.)
    #dLdx = -1.0*dVdy
    #dLdy = dVdx
    #tangent_out = tuple(x_dot*dVdx + y_dot*dVdy,x_dot*dLdx + y_dot*dLdy)
    return primal_out, x_dot*dVdx + y_dot*dVdy

In [85]:
dfdx=grad(f,argnums=0)
dfdxv=vmap(grad(f,argnums=(0,1)),(0,None),0)

In [89]:
xarr=jnp.array([1.0,2.0])
#dfdxv(xarr,0.1)


In [74]:
%timeit g(1.0,0.1)

3.12 ms ± 140 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [76]:
from jax import vmap
dgdx = vmap(g, (0, None), 0)

In [77]:
import matplotlib.pyplot as plt
xarr=jnp.linspace(1,10,10)
plt.plot(xarr,dgdx(xarr,0.1))


Exception: The numpy.ndarray conversion method __array__() was called on the JAX Tracer object Traced<ShapedArray(complex64[])>with<BatchTrace(level=1/1)>
  with val = DeviceArray([ 1.       +0.1j,  2.       +0.1j,  3.       +0.1j,
                           4.       +0.1j,  5.       +0.1j,  6.       +0.1j,
                           7.0000005+0.1j,  8.       +0.1j,  9.       +0.1j,
                          10.       +0.1j], dtype=complex64)
       batch_dim = 0.

This error can occur when a JAX Tracer object is passed to a raw numpy function, or a method on a numpy.ndarray object. You might want to check that you are using `jnp` together with `import jax.numpy as jnp` rather than using `np` via `import numpy as np`. If this error arises on a line that involves array indexing, like `x[idx]`, it may be that the array being indexed `x` is a raw numpy.ndarray while the indices `idx` are a JAX Tracer instance; in that case, you can instead write `jax.device_put(x)[idx]`.